In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install mtcnn
!pip install Pillow

## **Use:-**<br>
1) Code that will allow company to extract out and save (in company's database) only face of user A from rest all extracted faces from the pictures uploaded in his/her profile.<br>

2) To do so, after user A will upload his/her pic in the Usoon profile, this system will be asking him/her out of all extracted faces (out of each and every photo in hi/her profile), which one is his/her own photo.<br>

3) As per the answer typed in by the user A, the system will either saveor discard the face-cutouts.<br> 

4) These face cutouts of user A will be beneficial to match his/her presence in some photo uploaded by user B. This  in turn if validates True will save the user A in friend list of user B.

## **Features:-**<br>
1) Will extract all face-cutouts from every picture present in Usoon profile of 'user A'.<br>
2) Then like 'Google photos', the system will ask user A to verify which face-cutout is of him/her.<br>
3) The system during verification process will only allow user to input '1' (for 'yes') and '2' (for 'no').<br>
4) The system will provide feature to stop the verification process at any moment and resume it from the same point.<br>
5) The system will prevent the entry of pictures which have more than 5 faces detected in it and which dont have suitable format to be saved in storage.       



In [ ]:
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN
import os
import sys
import cv2
import shutil

In [ ]:
#this class will help to raise error if user tries to input anything apart from 'yes' or 'no'
class not_ok(Exception):
  pass

In [ ]:
#this function will help company to draw-out extracted faces, ask user to recognize his face cutout out of rest all and then 
#will either save or not save a particular face cutout according to user's reply. 
def draw_faces(imagename, result):
  with open("/content/drive/My Drive/erevna/Usoon_private_database/"+str(id)+"_counter.txt", 'a') as encoded_counter:
    encoded_counter.write(item+"\n")
  countt=0
  #load the image. Lets suppose this image consists of 3 faces.
  data = cv2.imread(imagename)
  #initialize an empty dictionary
  d={}
  #now as this image has 3 faces, so len(result) will be 3
  for i in range(len(result)):
    #so here i, for first run through 'for' loop will means first face cutout out of 3 face cutouts
    # get coordinates of box that would be enclosing 'i',i.e. first face cutout 
    x1, y1, width, height = result[i]['box']
    x2, y2 = x1 + width, y1 + height
    #kickstarting plotting of first face cutout
    pyplot.subplot(1, len(result), i+1)
    pyplot.axis('off')
    #above plotted first face cutout will be this label been printed 
    print("photo numbered",i+1)
    #now we will cut short our full image "data" into size of first face cutout's enclosing box
    imagee = data[y1:y2, x1:x2]
    face_cutout = cv2.resize(imagee,(160, 160))
    #doing manupulations with names 
    a=imagename.split('/')[7]
    aa=a.split('.')[0]
    b=company_data+aa+str("=")+str(i)+".jpg"
    #we save the company-made-path of first face cutout as a value with key "i",i.e. "0"
    #this will be later used to get reference of each face cutout in a particular and fixed order 
    d[i]=b    
    #this will save the cutouts in company's database
    try:
      cv2.imwrite(d[i], face_cutout)
    except Exception:
      continue
    else:
      pyplot.imshow(face_cutout)
      pyplot.show()
      print('Hey!')

  for i in range(len(result)):
    if os.path.exists(d[i]):
        while True:
          try:   
              #we will ask user which face cutout belongs to him                
              confirm = input("Photo numbered "+str(i+1)+" is you? ('1' for 'yes'/ '2' for 'no'):- ")
              for elem in confirm.split():
                  #if user tried to input anything other than 'yes'or 'no'in reply to our confirmation question,
                  #an error will be raised (which we already defined veryyyy above) saying that you can only reply 'yes' or 'no'
                  if not (elem=="1" or elem=="2"):
                    raise not_ok
                  #if user replied to our confirmation question in 'yes' or 'no', then we will break from here and will go to
                  #'else' present below 'except'
                  break
          except not_ok:
              print("Sorry, type in '1' for 'yes'/ '2' for 'no'.")
          else:
            if elem=="1":
              print("saved image")
              print("")
              print("")
            else:
              os.remove(d[i]) #remove part
              print("not saved image")
              print("")
              print("")

...........................................................................................................................................................................................................................................

**Control Unit**

In [ ]:
#here we our asking user to input his user ID 
id = input("please enter your Usoon ID:- ") #use 'renuka' as 'Usoon ID' for current time.

#location for company_data containing face cutouts of user
company_data = "/content/drive/My Drive/erevna/Usoon_private_database/"+str(id)+"2.0/"

#if there will not be any folder in company's database to store face-cutouts of user, then folder will be formed  
if not os.path.exists(company_data):
    os.makedirs(company_data)


#location for user_data containing whole image of (alone or with someone else)
user_data = os.listdir('/content/drive/My Drive/erevna/'+str(id)+'/pics')

#this 'names' list helps us to keep track of images which have been verified. Thus after resuming the process the process,
#it prevents system to again show images which have been already been verified (to be either '1' for 'yes'/ '2' for 'no')
try:
  names=[]
  with open("/content/drive/My Drive/erevna/Usoon_private_database/"+str(id)+"_counter.txt", 'r') as f:
    lines = f.readlines()
    for line in lines:
      linee=line.split('/')[7]
      lineee=linee.split('\n')[0]
      names.append(lineee)
except Exception:
  names=[]


#we take first image of user from 'user_data' 
for entry in user_data:
  #print(entry)
  #just pick-out name (no '.jpg')
  entryyy = entry.split('.')[0]
  #make full path of first image (came from 'user_data')
  item = '/content/drive/My Drive/erevna/'+str(id)+'/pics/'+ str(entry)
  # load that image from 'user_data'
  image_loadedd = cv2.imread(item)
  # create the detector, using default weights
  face_detector = MTCNN()
  # detect faces in that image
  faces = face_detector.detect_faces(image_loadedd)
  #print(len(faces))
  #then if number of faces present in that image(came from 'user_data') are less than 5(did this to avoid situation of confirming 22 face cutouts of being of user or not)
  #and if that image don't exist in 'company_data' (i.e if any of faces from that image has still not been saved in 'company_data),
  #then we will call function 'draw_faces' to our next job
  if len(faces)<5 and entry not in names:
    #to see what is this function, scroll-up
    draw_faces(item, faces)